In [ ]:
# | default_exp declination

In [ ]:
# | hide
from nbdev.showdoc import *
from fastcore.test import *

In [ ]:
# | export
from math import cos, sin, atan

In [ ]:
# | export


def declination(doy: int):  # julian day (day of the year)
    "Calculate declination of sun (radians ? ) for a given julian day (DOY)"

    # Hervé's formula for solar declination

    # Sin(23.5*pi/180), 23.5 = Earth declination

    # Constans ----------------------------------------------------------------
    c1 = 0.398749068925246

    c2 = 2 * 3.1416 / 365

    # date of spring
    c3 = 80

    x = c1 * sin((doy - c3) * c2)  # ;

    # Return declination ------------------------------------------------------
    return atan(x / ((1 - x * x) ^ 0.5))

In [ ]:
def potential_par(time_of_day: float, lat: float, doy: float):
    "Determine potential for a given place and date /used to determine cloud cover return potential par in W.m2"

    # Constants ----------------------------------------------------------
    diffuse_fraction = 0.1
    solar_constant = 2084
    attenuation_coef = -0.174353387144778

    decl = declination(doy)

    pn = -cos(lat * pi / 180)
    pz = sin(lat * pi / 180)
    h_rad = (time_of_day - 6) * 3.1416 / 12
    se = cos(h_rad) * cos(decl)
    sn = -pz * sin(h_rad) * cos(decl) - pn * sin(decl)
    sz = -pn * sin(h_rad) * cos(decl) + pz * sin(decl)
    alt = atan(sz / ((se * se + sn * sn) ^ 0.5))
    azi = 3.1416 + atan(se / sn)

    # azi[sn > 0] = azi[sn > 0] + 3.1416 I dont understand the meaning of this
    if sn > 0:
        azi = azi + pi

    pfd = solar_constant * exp(attenuation_coef / sin(alt))

    # pfd[alt < 0] = 0 I dont understand the meaning of this
    dpfd = diffuse_fraction * pfd
    # dpfd[alt<0] = 0 I dont understand the meaning of this

    return dpfd + pfd * sin(alt)